In [9]:
from herbie import Herbie
from datetime import datetime, timedelta
import math
from herbie import FastHerbie
import pandas as pd
    
file_name = '/home/bran/SNOWPACK/input/Tioga/Tioga_test.smet'
data_index = 1 #index is station lat/long, starts at 0
start_time_str = '2023-10-25 00:00:00'
ss = ":TMP:2 m|:RH:2 m|:WIND:10 m|:DSWRF:surface|:DLWRF:surface|:ULWRF:surface|USWRF:surface|:APCP:.*:(?:0-1[1-9]\d*-\d+) hour"
ss1 = ":TMP:2 m|:RH:2 m" # 2m readings
ss2 = ":WIND:10 m" # 10m readings
ss3 = ":GUST:surface|:DSWRF:surface|:DLWRF:surface|:ULWRF:surface|USWRF:surface|:APCP:surface:(?:0-1|[1-9]\d*-\d+)*" # surf readings minus precip
#ss4 = f":APCP:surface:{i-1}-{i} h*" #hourly precip
ll = [(-119.02,37.64),(-119.25,37.91),(-118.73,37.45)] # lat/long order: CUES, Tioga, Rock_Creek
maxrun = 2 # to get 48hr model: maxrun = 49

DATES = pd.date_range(
    start = start_time_str,
    periods=1,
    freq="24H")

fxx = range(0,maxrun)

FH = FastHerbie(DATES, model="hrrr", fxx=fxx)
FH
ds1 = FH.xarray(ss1, remove_grib=False)
y1=ds1.herbie.nearest_points(points=ll)
print('querying data for 2 m')
y1_d = y1.to_dict()
ds2 = FH.xarray(ss2, remove_grib=False)
y2=ds2.herbie.nearest_points(points=ll)
print('querying data for 10 m')
y2_d = y2.to_dict()
ds3 = FH.xarray(ss3, remove_grib=False)
y3=ds3.herbie.nearest_points(points=ll)
print('querying data for surface')
y3_d = y3.to_dict()


#ss4 = ":APCP:surface:(?:0-1|[1-9]\d*-\d+)*" #hourly precip
#
#print('querying data for precip')
#ds4 = FH.xarray(ss4, remove_grib=False)
#y4=ds4.herbie.nearest_points(points=ll)
#y4_d = y4.to_dict()

for i in range(maxrun):
    dt = datetime.fromisoformat(start_time_str)
    dt_query_str = dt.strftime('%Y-%m-%d %H:%M')
    output_timestamp = dt + timedelta(hours=i)
    output_timestamp_str = output_timestamp.strftime('%Y-%m-%dT%H:%M')
    hours_offset = i
    #H = Herbie(start_time_str,model="hrrr",fxx=hours_offset,save_dir="~/ESAC_herbiedata")
    #print(H.inventory(searchString=":TMP:2 m|:RH:2 m|:WIND:10 m|:.GRD:10 m|:DSWRF:surface|:DLWRF:surface|:ULWRF:surface|USWRF:surface|:APCP:surface"))
    # not sure about winds
    
    
    append_data(y1_d, y2_d, y3_d, output_timestamp_str)

/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "t2m".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "r2".
  ds = ds.metpy.assign_y_x()


querying data for 2 m


/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "si10".
  ds = ds.metpy.assign_y_x()


querying data for 10 m
querying data for surface
<class 'list'>
63
63
<class 'list'>
63
63


/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "tp".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "gust".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "dswrf".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "uswrf".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordinate present for variable  "dlwrf".
  ds = ds.metpy.assign_y_x()
/home/bran/anaconda3/lib/python3.11/site-packages/herbie/accessors.py:264: UserWarning: More than one time coordin

In [8]:
def append_data(y1_d, y2_d, y3_d, output_timestamp_str):
    t2m = y1_d['data_vars']['t2m']['data'][data_index][i]
    r2 = y1_d['data_vars']['r2']['data'][data_index][i]
    wind10 = y2_d['data_vars']['si10']['data'][data_index][i]
    gust = y3_d['data_vars']['gust']['data'][data_index][i]
    dsw = y3_d['data_vars']['dswrf']['data'][data_index][i]
    dlw = y3_d['data_vars']['dlwrf']['data'][data_index][i]
    ulw = y3_d['data_vars']['ulwrf']['data'][data_index][i]
    usw = y3_d['data_vars']['uswrf']['data'][data_index][i]
    hrprecip = y3_d['data_vars']['tp']['data'][data_index][i]
        
    t2m_formatted = round(t2m, 1) #2nd parameter is number of decimals
    r2_formatted = round(r2)
    gust_formatted = round(gust, 1)
    wind10_formatted = round(wind10, 1)
    dsw_formatted = round(dsw)
    dlw_formatted = round(dlw)
    ulw_formatted = round(ulw)
    usw_formatted = round(usw)
    hrprecip_formatted = round(hrprecip, 1)
    
    new_line_str = f"{output_timestamp_str} {t2m_formatted} {r2_formatted} {gust_formatted} {wind10_formatted} {dsw_formatted} {dlw_formatted} {ulw_formatted} {usw_formatted} {hrprecip_formatted} 0.0\n"
    with open(file_name, 'r') as f:
        lines = f.readlines()
        print(type(lines))
        print(len(lines))
        new_lines = lines
        found= False
        for idx, line in enumerate(lines):
            if line[:16] == output_timestamp_str:
                new_lines[idx] = new_line_str
                found = True
                break
        if not found:
            new_lines += [new_line_str]
    print(len(new_lines))
    with open(file_name, 'w') as f:
        f.writelines(new_lines)
        
print("done")

done


In [7]:
hm2 = FH.inventory(searchString=ss2)
print(hm2)

    grib_message  start_byte  end_byte              range reference_time  \
0             77    44266461  46409933  44266461-46409933     2023-10-25   
1             78    46409933  48791548  46409933-48791548     2023-10-25   
2             79    48791548  49975092  48791548-49975092     2023-10-25   
3             77    51205669  53349141  51205669-53349141     2023-10-25   
4             78    53349141  55730756  53349141-55730756     2023-10-25   
..           ...         ...       ...                ...            ...   
67            78    57877213  60020685  57877213-60020685     2023-10-25   
68            79    60020685  61218136  60020685-61218136     2023-10-25   
69            77    55735243  58116858  55735243-58116858     2023-10-25   
70            78    58116858  60260330  58116858-60260330     2023-10-25   
71            79    60260330  61463785  60260330-61463785     2023-10-25   

            valid_time variable              level        forecast_time  \
0  2023-10-2

In [7]:
from herbie import Herbie
from datetime import datetime, timedelta
import math
from herbie import FastHerbie
import pandas as pd
    
file_name = '/home/bran/SNOWPACK/input/Tioga/Tioga_test.smet'
data_index = 1 #index is station lat/long, starts at 0
start_time_str = '2023-10-25 00:00:00'
# ss2 = ":WIND:10 m|:GUST:10 m" # 10m readings
ss3 = ":APCP:surface:(?:0-1|[1-9]\d*-\d+)*" # surf readings minus precip
maxrun = 2 # to get 48hr model: maxrun = 49

DATES = pd.date_range(
    start = start_time_str,
    periods=1,
    freq="24H")

fxx = range(0,maxrun)

FH = FastHerbie(DATES, model="hrrr", fxx=fxx)
FH
# ds2 = FH.xarray(ss2, remove_grib=False)
# print(ds2)
# print('querying data for 10 m')
ds3 = FH.xarray(ss3, remove_grib=False)
print(ds3)
print('querying data for surface')


<xarray.Dataset>
Dimensions:              (step: 2, y: 1059, x: 1799)
Coordinates:
    time                 datetime64[ns] 2023-10-25
  * step                 (step) timedelta64[ns] 00:00:00 01:00:00
    surface              float64 0.0
    latitude             (y, x) float64 21.14 21.15 21.15 ... 47.86 47.85 47.84
    longitude            (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time           (step) datetime64[ns] 2023-10-25 2023-10-25T01:00:00
Dimensions without coordinates: y, x
Data variables:
    tp                   (step, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    gribfile_projection  object None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 sfc
    description:  